In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import math
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics
from sklearn.model_selection import KFold
import time

In [2]:
#Splitting and Vectorization
def DataPrepocessing(X_train,y_train):
    corpus=X_train
    vectorizer = CountVectorizer()
    x = vectorizer.fit_transform(corpus)
    col=vectorizer.get_feature_names()
    trainData=pd.DataFrame(x.toarray(),columns=col,index=y_train.index)
    trainData['output']=y_train
    return [trainData,col,vectorizer]

#Training
def Training(trainData,col):
    outList=trainData['output'].value_counts().keys()
    data=dict([(key,[]) for key in outList])
    totalWords=dict([(key,[]) for key in outList])
    DataSeries=dict([(key,[]) for key in outList])
    ProbOutput=dict([(key,[]) for key in outList])
    size=trainData.shape[0]
    Total=len(col)
    trainingDict=dict([(key,dict([(keys, 0) for keys in outList])) for key in col])
    for i in outList:
        data[i]=((trainData[trainData['output']==i]))
        del data[i]['output']
        ProbOutput[i]=data[i].shape[0]/size
        DataSeries[i]=data[i].sum()
        totalWords[i]=(DataSeries[i]).sum()
        ind=DataSeries[i].index
        for j in ind:
            trainingDict[j][i]=(DataSeries[i][j]+1)/(totalWords[i]+Total)
    return [trainingDict,ProbOutput,outList]
#Testing
def Testing(X_test,trainingDict,ProbOutput,outList,vectorizer):
    corpus=X_test
    val=vectorizer.transform(corpus)
    col=vectorizer.get_feature_names()
    ind=range(0,y_test.shape[0])
    testData=pd.DataFrame(val.toarray(),columns=col,index=ind)
    predict=[]
    for i in ind:
        q=np.nonzero(testData.iloc[i])
        length=len(q[0])
        Pred=dict([(key,1) for key in outList])
        for j in range(0,length):
            for k in Pred.keys():
                Pred[k]=Pred[k]*trainingDict[col[q[0][j]]][k]
        for j in Pred.keys():
            Pred[j]=Pred[j]*ProbOutput[j]
        predict.append(max(Pred,key=Pred.get))
    return predict

In [6]:
#Load Data
df = pd.read_csv('https://raw.githubusercontent.com/justmarkham/pycon-2016-tutorial/master/data/sms.tsv',header=None,delimiter = '\t')

#Declaring Input and Output
X=df[1]
y=df[0]

#Cross Validation creating an object of KFold which shall divide the dataset in 5 groups and apply our algorithm on each and every group
kf=KFold(n_splits=5,shuffle=True,random_state=42)
Predict=[None]*df.shape[0]
for train_index,test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    [trainData,col,vectorizer]=DataPrepocessing(X_train,y_train)
    [trainingDict,ProbOutput,outList]=Training(trainData,col)
    proxy=Testing(X_test,trainingDict,ProbOutput,outList,vectorizer)
    for i in range(0,len(proxy)):
        Predict[test_index[i]]=proxy[i]
acc=metrics.accuracy_score(y, Predict)*100
print("The accuracy of this Algorithm with Cross Validation is "+str(acc))

C:\Users\Trivikram\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:61: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
  return bound(*args, **kwds)


[   8   12   15 ... 5561 5565 5567]


C:\Users\Trivikram\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:61: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
  return bound(*args, **kwds)


[  14   19   24 ... 5553 5569 5571]


C:\Users\Trivikram\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:61: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
  return bound(*args, **kwds)


[   0    6    7 ... 5554 5563 5566]


C:\Users\Trivikram\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:61: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
  return bound(*args, **kwds)


[   1    2   10 ... 5557 5559 5570]


C:\Users\Trivikram\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:61: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
  return bound(*args, **kwds)


[   3    4    5 ... 5562 5564 5568]
The accuracy of this Algorithm with Cross Validation is 98.7078248384781
